In [1]:
!pip install requests transformers pandas datetime vaderSentiment requests yfinance finnhub-python
!pip install nltk
!pip install vaderSentiment
!pip install yahoo-fin
!pip install finvizfinance
!pip install "websockets>=11.0"
!pip install yfinance yahoo_fin nltk textblob matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 6.9 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=4e0d69e3881c8b5264cc549111f21b2f9f6e28006bef57fcd8d484f5b72d76ab
  Stored 

In [2]:
# Importing necessary libraries for stock data, time handling, sentiment analysis, and file operations
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import pytz
from yahoo_fin import news as yahoo_fin_news
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

# Set up sentiment tools and time zone
nltk.download('vader_lexicon')  # Download VADER lexicon for sentiment scoring
vader = SentimentIntensityAnalyzer()  # Create VADER analyzer
tz = pytz.timezone('US/Eastern')  # Set timezone to Eastern (for consistent timestamps)
now = datetime.now(tz)  # Current time (Eastern)
start_window = now - timedelta(days=90)  # We only want news articles from the past 90 days

# Upload company tickers file (user must upload manually in Colab)
from google.colab import files
uploaded = files.upload()

# Read and clean ticker list from uploaded text file
with open("company_tickers.txt", "r") as f:
    content = f.read()
TICKERS = [ticker.strip().replace('"', '') for ticker in content.split(',')]  # remove spaces and quotes

# Define time intervals to compare stock price movements relative to article time
INTERVALS = {
    "1 Hour Ago": timedelta(hours=1),
    "4 Hours Ago": timedelta(hours=4),
    "Previous Day Close": timedelta(days=1),
    "7 Days Ago": timedelta(days=7)
}

# Class to handle news collection and sentiment/price analysis for a given ticker
class NewsAnalyzer:
    def __init__(self, ticker):
        self.ticker = ticker
        self.stock = yf.Ticker(ticker)  # Create yfinance object for the stock

    # Function to fetch latest news articles related to the ticker
    def fetch_articles(self):
        try:
            # Go through all articles and keep only those published within the 90-day window
            all_articles = [
                {
                    "Ticker": self.ticker,
                    "Title": art["title"],
                    "Summary": art["summary"],
                    "URL": art["link"],
                    "Datetime": datetime.strptime(art["published"], "%a, %d %b %Y %H:%M:%S %z").astimezone(tz)
                }
                for art in yahoo_fin_news.get_yf_rss(self.ticker)
                if start_window <= datetime.strptime(art["published"], "%a, %d %b %Y %H:%M:%S %z").astimezone(tz) <= now
            ]
            # Sort articles by date (newest first) and return the top 3 only
            all_articles.sort(key=lambda x: x["Datetime"], reverse=True)
            return all_articles[:3]
        except Exception as e:
            print(f"Error fetching news for {self.ticker}: {e}")
            return []

    # Function to get the stock's price closest to a given datetime
    def get_price_near(self, timepoint):
        try:
            # Use hourly price history around the target time
            hist = self.stock.history(
                start=timepoint.strftime('%Y-%m-%d'),
                end=(timepoint + timedelta(days=1)).strftime('%Y-%m-%d'),
                interval='1h'
            )
            if not hist.empty:
                # Find the closest price timestamp to the news time
                hist['Delta'] = abs(hist.index - timepoint)
                return round(hist.sort_values('Delta').iloc[0]['Close'], 2)
        except:
            return None
        return None

    # Main function to analyze a single news article: sentiment + price movement
    def analyze_article(self, article):
        base_time = article["Datetime"]  # When the news was published
        base_price = self.get_price_near(base_time)  # Get price at that moment
        if base_price is None:
            return None  # Skip if no price available

        prices = {}      # To store historical prices
        pct_changes = {} # To store % change from article time to each past point

        # Loop through each defined time interval (1h, 4h, 1d, 7d)
        for label, delta in INTERVALS.items():
            time_check = base_time - delta
            past_price = self.get_price_near(time_check)
            prices[label] = past_price if past_price else 'N/A'
            # If price data is valid, compute % change
            pct_changes[label + " % Change"] = (
                round(((base_price - past_price) / past_price) * 100, 2) if isinstance(past_price, (int, float)) else 'N/A'
            )

        # Combine everything into one result dictionary
        return {
            **article,
            "News Age (Days)": (now - base_time).days,
            "News Time": base_time,
            "Current Price": base_price,
            **prices,
            **pct_changes,
            # Sentiment analysis
            "Headline Sentiment": round(TextBlob(article["Title"]).sentiment.polarity, 2),
            "Summary Sentiment": round(vader.polarity_scores(article["Summary"])["compound"], 2)
        }

# Run analysis on all tickers and save results into a CSV file
def run_full_analysis():
    full_dataset = []  # Collect all articles and results here
    for ticker in TICKERS:
        analyzer = NewsAnalyzer(ticker)
        articles = analyzer.fetch_articles()
        for article in articles:
            result = analyzer.analyze_article(article)
            if result:
                full_dataset.append(result)

    # Convert results to dataframe
    df = pd.DataFrame(full_dataset)
    df.to_csv("Companies_Sentiment_Analysis_7_Days_Track.csv", index=False)
    print("Saved to Companies_Sentiment_Analysis_7_Days_Track.csv")
    return df

# Execute the full analysis
df_result = run_full_analysis()
print(df_result.head())  # Preview first few rows of the result

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


Saving company_tickers.txt to company_tickers (1).txt


ERROR:yfinance:$CSCO: possibly delisted; no price data found  (1h 2025-08-03 -> 2025-08-04)
ERROR:yfinance:$CSCO: possibly delisted; no price data found  (1h 2025-08-03 -> 2025-08-04)
ERROR:yfinance:$CSCO: possibly delisted; no price data found  (1h 2025-08-03 -> 2025-08-04)
ERROR:yfinance:$NET: possibly delisted; no price data found  (1h 2025-08-03 -> 2025-08-04)
ERROR:yfinance:$NET: possibly delisted; no price data found  (1h 2025-08-03 -> 2025-08-04)
ERROR:yfinance:$HMC: possibly delisted; no price data found  (1h 2025-08-03 -> 2025-08-04)
ERROR:yfinance:$HMC: possibly delisted; no price data found  (1h 2025-08-03 -> 2025-08-04)
ERROR:yfinance:$HMC: possibly delisted; no price data found  (1h 2025-08-03 -> 2025-08-04)
ERROR:yfinance:$CRM: possibly delisted; no price data found  (1h 2025-08-03 -> 2025-08-04)
ERROR:yfinance:$CRM: possibly delisted; no price data found  (1h 2025-08-03 -> 2025-08-04)
ERROR:yfinance:$CRM: possibly delisted; no price data found  (1h 2025-08-03 -> 2025-08-

Saved to Companies_Sentiment_Analysis_7_Days_Track.csv
  Ticker                                              Title  \
0   CSCO  SES Space & Defense Awarded Sustainment Tactic...   
1   CSCO  AppOmni Innovations Combat Rising Security Ris...   
2   CSCO  Cisco Systems Stock: Is Wall Street Bullish or...   
3    NET  Is Most-Watched Stock Cloudflare, Inc. (NET) W...   
4    NET  Three Elite Growth Companies With Significant ...   

                                             Summary  \
0  RESTON, Va., August 04, 2025--SES’s wholly-own...   
1  LAS VEGAS, August 04, 2025--From Black Hat, Sa...   
2  Even though Cisco Systems has outpaced the bro...   
3  Cloudflare (NET) has been one of the stocks mo...   
4  As the U.S. stock market grapples with recent ...   

                                                 URL  \
0  https://finance.yahoo.com/news/ses-space-defen...   
1  https://finance.yahoo.com/news/appomni-innovat...   
2  https://www.barchart.com/story/news/33856248/c...   
3  ht

In [3]:
from google.colab import files
files.download("Companies_Sentiment_Analysis_7_Days_Track.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>